In [64]:
from randomized_searcher import *
from qcg.generators import gen_supremacy
from qiskit.converters import circuit_to_dag, dag_to_circuit
from qiskit.tools.visualization import dag_drawer
import randomized_searcher as r_s
        

circ = gen_supremacy(3,3,8,'71230456')
stripped_circ = r_s.circ_stripping(circ)
dag = circuit_to_dag(stripped_circ)
for vertex in dag.topological_op_nodes():
    print(id(vertex))

Generating 3x3, 1+8+1 supremacy circuit
4882120488
4882119368
4878083184
4878083744
4878083968
4878084192
4878084416
4878198824
4878198880
4878199216
4878199720
4878196976


In [2]:
print(graph.verts)

{'qreg[1]0,qreg[2]0': Counter({'qreg[0]0,qreg[1]1': 1, 'qreg[2]1,qreg[5]1': 1}), 'qreg[0]0,qreg[1]1': Counter({'qreg[0]1,qreg[3]1': 1, 'qreg[1]0,qreg[2]0': 1, 'qreg[1]2,qreg[4]3': 1}), 'qreg[4]0,qreg[7]0': Counter({'qreg[3]0,qreg[4]1': 1, 'qreg[7]1,qreg[8]0': 1}), 'qreg[3]0,qreg[4]1': Counter({'qreg[0]1,qreg[3]1': 1, 'qreg[4]0,qreg[7]0': 1, 'qreg[4]2,qreg[5]0': 1}), 'qreg[0]1,qreg[3]1': Counter({'qreg[0]0,qreg[1]1': 1, 'qreg[3]0,qreg[4]1': 1, 'qreg[3]2,qreg[6]1': 1}), 'qreg[4]2,qreg[5]0': Counter({'qreg[3]0,qreg[4]1': 1, 'qreg[1]2,qreg[4]3': 1, 'qreg[2]1,qreg[5]1': 1}), 'qreg[1]2,qreg[4]3': Counter({'qreg[0]0,qreg[1]1': 1, 'qreg[4]2,qreg[5]0': 1}), 'qreg[2]1,qreg[5]1': Counter({'qreg[1]0,qreg[2]0': 1, 'qreg[4]2,qreg[5]0': 1, 'qreg[5]2,qreg[8]1': 1}), 'qreg[7]1,qreg[8]0': Counter({'qreg[4]0,qreg[7]0': 1, 'qreg[6]0,qreg[7]2': 1, 'qreg[5]2,qreg[8]1': 1}), 'qreg[5]2,qreg[8]1': Counter({'qreg[2]1,qreg[5]1': 1, 'qreg[7]1,qreg[8]0': 1}), 'qreg[6]0,qreg[7]2': Counter({'qreg[3]2,qreg[6]1': 1, '

In [3]:
print(graph.edges)

[('qreg[1]0,qreg[2]0', 'qreg[2]1,qreg[5]1'), ('qreg[0]0,qreg[1]1', 'qreg[0]1,qreg[3]1'), ('qreg[0]0,qreg[1]1', 'qreg[1]0,qreg[2]0'), ('qreg[0]0,qreg[1]1', 'qreg[1]2,qreg[4]3'), ('qreg[4]0,qreg[7]0', 'qreg[7]1,qreg[8]0'), ('qreg[3]0,qreg[4]1', 'qreg[4]0,qreg[7]0'), ('qreg[3]0,qreg[4]1', 'qreg[4]2,qreg[5]0'), ('qreg[0]1,qreg[3]1', 'qreg[3]0,qreg[4]1'), ('qreg[0]1,qreg[3]1', 'qreg[3]2,qreg[6]1'), ('qreg[1]2,qreg[4]3', 'qreg[4]2,qreg[5]0'), ('qreg[2]1,qreg[5]1', 'qreg[4]2,qreg[5]0'), ('qreg[2]1,qreg[5]1', 'qreg[5]2,qreg[8]1'), ('qreg[5]2,qreg[8]1', 'qreg[7]1,qreg[8]0'), ('qreg[6]0,qreg[7]2', 'qreg[7]1,qreg[8]0'), ('qreg[3]2,qreg[6]1', 'qreg[6]0,qreg[7]2')]
